In [1]:
import numpy as np
import pandas as pd



In [2]:
# Pure Function

In [3]:
def check_purity(data):
    class_column = data[:, -1]
    unique_classes = np.unique(class_column)
    if len(unique_classes) == 1:
        return True
    else:
        return False

In [4]:
# Classify Data

In [5]:
def classify_data(data):
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    return classification

In [6]:
# Potential Splits #not applicable to me but still following

In [7]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):        # excluding the last column which is the label
        potential_splits[column_index] = [0, 1]
    
    return potential_splits

In [8]:
# Split Data

In [9]:
def split_data(data, split_column):
    
    split_column_values = data[:, split_column]

    data_left = data[split_column_values == 0]
    data_right = data[split_column_values == 1]
    
    return data_left, data_right

In [10]:
# Lowest overall entropy

In [11]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)
    
    probabilities = counts / counts.sum() * 1.0
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [12]:
def calculate_overall_entropy(data_left, data_right):
    
    n = len(data_left) + len(data_right)
    p_data_left = len(data_left) / n
    p_data_right = len(data_right) / n

    overall_entropy = (p_data_left * calculate_entropy(data_left) + p_data_right * calculate_entropy(data_right))
    
    return overall_entropy

In [13]:
# Determine best split

In [14]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    best_split_column = 0
    for column_index in potential_splits:
        data_left, data_right = split_data(data, split_column=column_index)
        current_overall_entropy = calculate_overall_entropy(data_left, data_right)

        if current_overall_entropy <= overall_entropy:
            overall_entropy = current_overall_entropy
            best_split_column = column_index
            
    return best_split_column

In [15]:
def decision_tree_algorithm(df, counter=0, min_samples=0, max_depth=10):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df           
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        return classification
    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column = determine_best_split(data, potential_splits)
        data_left, data_right = split_data(data, split_column)
        # print(counter, split_column, data_left, data_right)
        
        # instantiate sub-tree
        feature_name = COLUMN_HEADERS[split_column]
        question = "{}".format(feature_name)
        sub_tree = {question: []}
        
        # find answers (recursion)
        no_answer = decision_tree_algorithm(data_left, counter, min_samples, max_depth)
        yes_answer = decision_tree_algorithm(data_right, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(no_answer)
            sub_tree[question].append(yes_answer)
        
        return sub_tree

In [23]:
df = pd.read_csv("/Users/nimish/Documents/ML/temp_dataset/training_set.csv")
dt = df.values
print(df[df['Class']==1])
tree = decision_tree_algorithm(df)
print(tree)

    A  B  C  Z  Class
7   1  1  1  0      1
8   0  0  0  1      1
9   0  0  1  1      1
10  0  1  0  1      1
11  0  1  1  1      1
12  1  0  0  1      1
13  1  0  1  1      1
14  1  1  0  1      1
15  1  1  1  1      1
{'Z': [{'C': [0, {'B': [0, {'A': [0, 1]}]}]}, 1]}


In [17]:
test_df = pd.read_csv("/Users/nimish/Documents/ML/temp_dataset/test_set.csv")
example = test_df.iloc[4]
example

A        0
B        0
C        0
Z        1
Class    1
Name: 4, dtype: int64

In [18]:
def classify_example(example, tree):
    question = list(tree.keys())[0]

    # ask question
    if example[question] == 0:
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [19]:
example = test_df.iloc[0]
solution = classify_example(example, tree)
print(solution)

1


In [20]:
# Accuracy

In [21]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["Class"]
    
    accuracy = df["classification_correct"].mean() * 100
    
    return accuracy

In [22]:
accuracy = calculate_accuracy(test_df, tree)
accuracy

80.0